# Script description
### Input
* MongoDB database provided by Michiel Aernouts. This database contains metadata of the Belair sensors installed on bpost vans using the LoRa network. 
* This database contains the following elements:
    * _id: this is a unique ID generated for this packet 
    * device: unique ID for the device that sent the packet 
    * gateways: a list of the gateways that picked up the packet with their respective parameters (max 3) 
    * gpsLat and gpsLon: location 
    * hdop: measure for the correctness of the location 
    * rx_time: time of reception of the packet 
    * seqNumber: sequence number for the packets 
    * spfact: Spreading factor

### Plots
One of the things that was noticed in the past is that the quality of the LoRa network depends on the location in the city. Another thing that needs to be investigated is whether or not the signal quality varies over time. The signal quality of the LoRa network is related to the spreading factor. The following plots are made to investigate spatial and temporal influences:
* Plot of 1000 random samples to get a first idea
* Choropleth of the average spreading factor per district of Antwerp
* Choropleth of the average spreading factor based on 3 differently sized grid structures of Antwerp

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Create a Mongo Client and load data

In [2]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
import folium
import json
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
from matplotlib import pyplot as plt
import time
from datetime import datetime
from datetime import timedelta  
from selenium import webdriver
from scipy import special, optimize
from IPython.display import Math
import math
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import export_png

In [3]:
print("Pymongo version: " + pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.Belair
collection = db.data

# Extract data, sort by timestamp and reset indexes
df = pd.DataFrame(list(collection.find())).reset_index(drop=True)
print(len(df))
df.head()

Pymongo version: 3.7.2
193008


,_id,device,gateways,gpsLat,gpsLon,hdop,rx_time,seqNumber,spfact
0,5c17af0d4fa20f164de4b0bb,3432333852378918,"[{'rssi': -93, 'esp': -93.461838, 'snr': 10, '...",51.194618,4.418419,0.66,2018-12-17 14:13:33.410,54916,7
1,5c17af2d4fa20f164de4b0c7,3432333852378918,"[{'rssi': -104, 'esp': -104.331955, 'snr': 11,...",51.194618,4.418417,0.69,2018-12-17 14:14:04.616,54917,7
2,5c17af3c4fa20f164de4b0cb,3432333855378A18,"[{'rssi': -107, 'esp': -111.124428, 'snr': -2,...",51.215408,4.413910,0.67,2018-12-17 14:14:19.768,15653,7
3,5c17af4d4fa20f164de4b0d4,3432333852378918,"[{'rssi': -107, 'esp': -107.973228, 'snr': 6, ...",51.194614,4.418419,0.74,2018-12-17 14:14:36.499,54918,7
4,5c17af604fa20f164de4b0df,3432333851378918,"[{'rssi': -114, 'esp': -119.455406, 'snr': -4,...",51.241817,4.430120,0.66,2018-12-17 14:14:56.056,48522,7


# Plot 1 - 1000 random samples

In [4]:
def color_producer(spf):
    if spf == 7:
        return 'darkgreen'
    elif spf == 8:
        return 'green'
    elif spf == 9:
        return 'yellow'
    elif spf == 10:
        return 'orange'
    elif spf == 11:
        return 'red'
    elif spf == 12:
        return 'darkred'
    else:
        return 'black'

In [5]:
def scatter_plot_spf(df):
    mp = folium.Map(location=[df['gpsLat'].mean(), df['gpsLon'].mean()], 
                    zoom_start=13)

    # Generate legend
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 100px; height: 150px; 
                     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                     ">&nbsp; <b> Legend </b> <br>
                     &nbsp; SPF 7 &nbsp; <i class="fa fa-circle" style="color:darkgreen"></i><br>
                     &nbsp; SPF 8 &nbsp; <i class="fa fa-circle" style="color:green"></i><br>
                     &nbsp; SPF 9 &nbsp; <i class="fa fa-circle" style="color:yellow"></i><br>
                     &nbsp; SPF 10 &nbsp; <i class="fa fa-circle" style="color:orange"></i><br>
                     &nbsp; SPF 11 &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
                     &nbsp; SPF 12 &nbsp; <i class="fa fa-circle" style="color:darkred"></i>
          </div>
         '''
    # Add legend to map
    mp.get_root().html.add_child(folium.Element(legend_html))
    
    # Extract latitude, longitude and spreading factor
    lat = list(df["gpsLat"])
    lon = list(df["gpsLon"])
    spf = list(df["spfact"])
    time = list(df["rx_time"])
    device = list(df["device"])

    fg = folium.FeatureGroup(name="Scatter plot Spreading Factor")
    
    # Add circles for every data point
    for lt, ln, sp, ti, dev in zip(lat, lon, spf, time, device):
        cim = folium.CircleMarker(location=[lt, ln],
                                radius = 6,
                                popup="SPF: " + str(sp) + "\n timestamp: " + str(ti) + "\n device: " + str(dev),
                                fill=True, # Set fill to True
                                fill_color=color_producer(sp),
                                color = color_producer(sp),
                                fill_opacity=0.7)
        fg.add_child(cim)

    # Show the map
    mp.add_child(fg)
    
    return mp

    # Save the map
    #mp.save(os.path.join('results', 'SPF_antwerp_random.html'))
    

In [6]:
df1 = df.sample(1000)
scatter_plot_spf(df1)

# Plot 2 - Average SPF on Choropleth per District

## Load areas (districts) of Antwerp
"antwerp.geojson" downloaded from: https://github.com/codeforamerica/click_that_hood/blob/master/public/data/antwerp.geojson 

In [9]:
antwerp = os.path.join('data', 'antwerp.geojson')
areas = gpd.read_file('data\\antwerp.geojson')
geo_json_data = json.load(open(antwerp))

## Generate dataframe containing the necessary information
* Assign each datapoint to an area with and Area ID and an Area Name
* Calculate per Area the average spreading factor
Info from:
* https://automating-gis-processes.github.io/CSC18/lessons/L4/point-in-polygon.html
* https://automating-gis-processes.github.io/2017/lessons/L3/geocoding.html

In [10]:
def calculate_spf_mean_area(df, areas):
    # Create new 'coordinates' column as tuples of Longitude and Latitude and transform them in Point
    df['coordinates'] = list(zip(df.gpsLon, df.gpsLat))
    df['coordinates'] = df['coordinates'].apply(Point)

    # create the GeoDataFrame by setting geometry with the coordinates created previously.
    gdf = gpd.GeoDataFrame(df, geometry='coordinates')
    gdf['Area Name']=np.nan
    gdf['Area ID']=np.nan

    # Calculate centroids
    areas['centroid']=areas.geometry.centroid

    #Assign name and ID to all the datapoints depending on their location
    for index, row in areas.iterrows():
        #print(row['name'])
        area = areas.loc[areas['name']==row['name']]
        area.reset_index(drop=True, inplace=True)
        pip_mask = gdf.within(area.loc[0, 'geometry'])
        pip_data = gdf.loc[pip_mask]
        gdf['Area Name'].loc[pip_mask] = row['name']
        gdf['Area ID'].loc[pip_mask] = row['cartodb_id']

    # Select the interesting features and Remove the datapoints that are not within the boundaries
    spf_data = gdf[['spfact', 'Area ID', 'Area Name']].dropna()

    # Calculate SPF means
    spf_means = spf_data.groupby('Area ID')['spfact'].mean().to_dict()
    
    return spf_means, spf_data

In [11]:
spf_means, spf_data = calculate_spf_mean_area(df,areas)

C:\Users\JeffG\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Plot the results on a folium map

In [12]:
def plot_areas_spf(geo_json_data, spf_means):
    # Define own color scale where areas without data are shown in grey
    def my_color_function(feature):
        """Maps low values to green and hugh values to red."""
        val = spf_means.get(feature['properties']['cartodb_id'],0)
        if val > 6.5 and val < 7.5 :
            return 'darkgreen'
        elif val > 7.5 and val < 8.5:
            return 'green'
        elif val > 8.5 and val < 9.5:
            return 'yellow'
        elif val > 9.5 and val < 10.5:
            return 'orange'
        elif val > 10.5 and val < 11.5:
            return 'red'
        elif val > 11.5 and val < 12.5:
            return 'darkred'
        else:
            return 'grey'
        
    mp = folium.Map(location=[51.214618, 4.418419], 
                    zoom_start=13)
    folium.GeoJson(
        geo_json_data,
        style_function=lambda feature: {
            'fillColor': my_color_function(feature),
            'color': 'black',
            'weight': 2,
            'dashArray': '5, 5'
        }
    ).add_to(mp)

    return mp

In [13]:
plot_areas_spf(geo_json_data, spf_means)

In [14]:
# Check occurrences of the different Spreading Factors for each area
freq = pd.crosstab(index=spf_data["Area Name"], 
                           columns=spf_data["spfact"])
freq

spfact,7,8,9,10,11,12
Area Name,,,,,,
Amandus - Atheneum,3570,641,366,224,121,51
Borgerhout Extra Muros,2337,474,470,260,96,25
Borgerhout Intra Muros Noord,298,118,133,178,133,75
Borgerhout Intra Muros Zuid,633,198,258,219,155,122
Brederode,11025,146,51,13,6,0
Centraal Station,4574,1308,1420,1687,977,346
Dam,4303,1888,1566,788,272,54
Deurne Dorp - Gallifort,5,0,0,0,0,0
Deurne Noord,114,21,25,4,4,3


# Plot 3 - Average SPF on Choropleth per grid area

## Create grid

In [15]:
def create_grid(bottom, left, top, right, nr_lat_steps, nr_lon_steps):
    import json

    feature_coll = """{{
      "type": "FeatureCollection",
      "features": [{0}]
    }}"""

    polygons_template = """{{
          "type": "Feature",
          "properties": {{
            "cartodb_id":{1}
          }},
          "geometry": {{
            "type": "Polygon",
            "coordinates": [{0}]
          }}
    }},"""

    # Template for the last polygon is slightly different
    last_polygons_template = """{{
          "type": "Feature",
          "properties": {{
            "cartodb_id":{1}
          }},
          "geometry": {{
            "type": "Polygon",
            "coordinates": [{0}]
          }}
    }}"""

    point_template = "[{0}, {1}], "
    # Template for the last point of a polygon is slightly different
    last_point_template = "[{0}, {1}] "

    polygons = ""


    _startbottom = bottom
    _startleft = left

    _endtop = top
    _endright = right

    lat_steps = nr_lat_steps
    lon_steps = nr_lon_steps

    lat_steplength = (_endright - _startleft)/lat_steps; #print(lat_steplength)
    lon_steplength = (_endtop - _startbottom)/lon_steps; #print(lon_steplength)

    _bottom = _startbottom
    _left = _startleft
    _top = _endtop
    _right = _endright

    id = 1

    for offset_lon in range(1, lon_steps+1):
        _top = _startbottom + lon_steplength * offset_lon
        for offset_lat in range(1, lat_steps+1):
            _right = _startleft + lat_steplength * offset_lat
            polygon = "[ " +  \
                point_template.format(_left, _bottom) + \
                point_template.format(_right, _bottom) + \
                point_template.format(_right, _top) + \
                point_template.format(_left, _top) + \
                last_point_template.format(_left, _bottom) + \
                "]"
            _left = _right
            if(offset_lon == lon_steps and offset_lat == lat_steps):
                polygons = polygons + last_polygons_template.format(polygon,id)
            else:
                polygons = polygons + polygons_template.format(polygon,id)
            id = id + 1
        _bottom = _top
        _left = _startleft

    print(feature_coll.format(polygons))
    with open('data/grid_{0}x{1}.json'.format(lat_steps,lon_steps), 'w') as outfile:
        json.dump(feature_coll.format(polygons), outfile)

In [16]:
# Choose boundaries for grid
bottom = 51.18
left = 4.35
top = 51.25
right = 4.47

# Choose nr of steps
nr_lat_steps = 10
nr_lon_steps = 10

In [17]:
create_grid(bottom, left, top, right, nr_lat_steps, nr_lon_steps)

{
      "type": "FeatureCollection",
      "features": [{
          "type": "Feature",
          "properties": {
            "cartodb_id":1
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [[ [4.35, 51.18], [4.362, 51.18], [4.362, 51.187], [4.35, 51.187], [4.35, 51.18] ]]
          }
    },{
          "type": "Feature",
          "properties": {
            "cartodb_id":2
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [[ [4.362, 51.18], [4.374, 51.18], [4.374, 51.187], [4.362, 51.187], [4.362, 51.18] ]]
          }
    },{
          "type": "Feature",
          "properties": {
            "cartodb_id":3
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [[ [4.374, 51.18], [4.385999999999999, 51.18], [4.385999999999999, 51.187], [4.374, 51.187], [4.374, 51.18] ]]
          }
    },{
          "type": "Feature",
          "properties": {
            "c

In [18]:
def assign_area_to_datapoint(gdf, areas):
    #Assign name and ID to all the datapoints depending on their location
    for index, row in areas.iterrows():
        #print(row['name'])
        area = areas.loc[areas['cartodb_id']==row['cartodb_id']]
        area.reset_index(drop=True, inplace=True)
        pip_mask = gdf.within(area.loc[0, 'geometry'])
        pip_data = gdf.loc[pip_mask]
        gdf['Area ID'].loc[pip_mask] = row['cartodb_id']
        
    return gdf

In [19]:
def calculate_spf_mean_grid(df, areas):
    # Create new 'coordinates' column as tuples of Longitude and Latitude and transform them in Point
    df['coordinates'] = list(zip(df.gpsLon, df.gpsLat))
    df['coordinates'] = df['coordinates'].apply(Point)

    # create the GeoDataFrame by setting geometry with the coordinates created previously.
    gdf = gpd.GeoDataFrame(df, geometry='coordinates')
    gdf['Area ID']=np.nan

    # Calculate centroids
    areas['centroid']=areas.geometry.centroid
    
    gdf = assign_area_to_datapoint(gdf,areas)

    #Assign name and ID to all the datapoints depending on their location
#     for index, row in areas.iterrows():
#         #print(row['name'])
#         area = areas.loc[areas['cartodb_id']==row['cartodb_id']]
#         area.reset_index(drop=True, inplace=True)
#         pip_mask = gdf.within(area.loc[0, 'geometry'])
#         pip_data = gdf.loc[pip_mask]
#         gdf['Area ID'].loc[pip_mask] = row['cartodb_id']

    # Select the interesting features and Remove the datapoints that are not within the boundaries
    spf_data = gdf[['spfact', 'Area ID']].dropna()

    # Calculate SPF means
    spf_means = spf_data.groupby('Area ID')['spfact'].mean().to_dict()
    
    return spf_means, spf_data, gdf

In [22]:
antwerp_grid = os.path.join('data', 'grid_{0}x{1}.json'.format(nr_lat_steps,nr_lon_steps))
geo_json_data = json.load(open(antwerp_grid))

#GeoJSON file needs to be generated by copying the json format to http://geojson.io and export it there as GeoJSON
grid_areas = gpd.read_file('data/grid_{0}x{1}.geojson'.format(nr_lat_steps,nr_lon_steps))
spf_means, spf_data, gdf = calculate_spf_mean_grid(df,grid_areas)

C:\Users\JeffG\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
plot_areas_spf(geo_json_data, spf_means)

# Plot 4 - Generate daily scatter plots for a certain district

In [ ]:
# Mark one area
area_name = 'Eilandje'
sample_area = areas.loc[areas['name']==area_name]
sample_area.reset_index(drop=True, inplace=True)

fig, ax = plt.subplots()
areas.plot(ax=ax, facecolor='gray');
sample_area.plot(ax=ax, facecolor='red');
plt.show();

In [ ]:
# Filter data for chosen area
pip_mask = gdf.within(sample_area.loc[0, 'geometry'])
pip_data = gdf.loc[pip_mask]
data = pip_data
data.head()

In [ ]:
def scatter_one_day(year, month, day, data, dist_name):
    # Set startview for map
    mp = folium.Map(location=[data['gpsLat'].mean() ,data['gpsLon'].mean() ], 
                    zoom_start=15)

    # Generate legend
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 100px; height: 150px; 
                     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                     ">&nbsp; <b> Legend </b> <br>
                     &nbsp; SPF 7 &nbsp; <i class="fa fa-circle" style="color:darkgreen"></i><br>
                     &nbsp; SPF 8 &nbsp; <i class="fa fa-circle" style="color:green"></i><br>
                     &nbsp; SPF 9 &nbsp; <i class="fa fa-circle" style="color:yellow"></i><br>
                     &nbsp; SPF 10 &nbsp; <i class="fa fa-circle" style="color:orange"></i><br>
                     &nbsp; SPF 11 &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
                     &nbsp; SPF 12 &nbsp; <i class="fa fa-circle" style="color:darkred"></i>
          </div>
         '''
    mp.get_root().html.add_child(folium.Element(legend_html))

    startDate = datetime(year, month, day, 0, 0); print(startDate)
    data_day = data[(data['rx_time'] > startDate) 
                                      & (data['rx_time'] < (startDate + timedelta(days=1)))]

    lat = list(data_day["gpsLat"])
    lon = list(data_day["gpsLon"])
    spf = list(data_day["spfact"])

    fg = folium.FeatureGroup(name=dist_name)
    for lt, ln, sp in zip(lat, lon, spf):
        cim = folium.CircleMarker(location=[lt, ln],
                                radius = 6,
                                popup="SPF: " +str(sp),
                                fill=True, # Set fill to True
                                fill_color=color_producer(sp),
                                color = color_producer(sp),
                                fill_opacity=0.7)
        fg.add_child(cim)

    mp.add_child(fg)

    delay=5

    #Save the map as an HTML file
    fn=str(dist_name)+startDate.strftime('%Y-%m-%d')+".html"
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    print(tmpurl)
    mp.save(fn)

    #Open a browser window...
    browser = webdriver.Chrome('C:\\Users\\JeffG\\Downloads\\chromedriver_win32\\chromedriver.exe')
    #..that displays the map...
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    #Grab the screenshot
    browser.save_screenshot(str(dist_name)+startDate.strftime('%Y-%m-%d')+'.png')
    #Close the browser
    browser.quit()

In [ ]:
# Test function for one day
scatter_one_day(2019,1,7,data,area_name)

In [ ]:
# Generate different scatter plots
#for i in range(1,32):
#    scatter_one_day(2018,12,i,data,area_name)

In [ ]:
# Show resulting gif
from IPython.display import Image
from IPython.display import display
display(Image(url='SPF_Eilandje_cumul.gif'))

# Plot 5 - Generate hourly scatter plots for a certain grid area

In [ ]:
# Mark one area
grid_id = 45
sample_area = grid_areas.loc[grid_areas['cartodb_id']==grid_id]
sample_area.reset_index(drop=True, inplace=True)

fig, ax = plt.subplots()
grid_areas.plot(ax=ax, facecolor='gray');
sample_area.plot(ax=ax, facecolor='red');
plt.show();

In [ ]:
# Filter data for chosen area
pip_mask = gdf.within(sample_area.loc[0, 'geometry'])
pip_data = gdf.loc[pip_mask]
data = pip_data
data.head()

In [ ]:
def scatter_one_hour(year, month, day, hour, data, dist_id):
    # Set startview for map
    mp = folium.Map(location=[data['gpsLat'].mean() ,data['gpsLon'].mean() ], 
                    zoom_start=15)

    # Generate legend
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 100px; height: 150px; 
                     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                     ">&nbsp; <b> Legend </b> <br>
                     &nbsp; SPF 7 &nbsp; <i class="fa fa-circle" style="color:darkgreen"></i><br>
                     &nbsp; SPF 8 &nbsp; <i class="fa fa-circle" style="color:green"></i><br>
                     &nbsp; SPF 9 &nbsp; <i class="fa fa-circle" style="color:yellow"></i><br>
                     &nbsp; SPF 10 &nbsp; <i class="fa fa-circle" style="color:orange"></i><br>
                     &nbsp; SPF 11 &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
                     &nbsp; SPF 12 &nbsp; <i class="fa fa-circle" style="color:darkred"></i>
          </div>
         '''
    mp.get_root().html.add_child(folium.Element(legend_html))

    startDate = datetime(year, month, day, hour, 0); print(startDate)
    data_hour = data[(data['rx_time'] > startDate) 
                                      & (data['rx_time'] < (startDate + timedelta(hours=1)))]

    lat = list(data_hour["gpsLat"])
    lon = list(data_hour["gpsLon"])
    spf = list(data_hour["spfact"])

    fg = folium.FeatureGroup(name=dist_id)
    for lt, ln, sp in zip(lat, lon, spf):
        cim = folium.CircleMarker(location=[lt, ln],
                                radius = 6,
                                popup="SPF: " +str(sp),
                                fill=True, # Set fill to True
                                fill_color=color_producer(sp),
                                color = color_producer(sp),
                                fill_opacity=0.7)
        fg.add_child(cim)

    mp.add_child(fg)

    delay=5

    #Save the map as an HTML file
    fn=str(dist_id)+startDate.strftime('%Y-%m-%d_%H%M')+".html"
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    print(tmpurl)
    mp.save(fn)

    #Open a browser window...
    browser = webdriver.Chrome('C:\\Users\\JeffG\\Downloads\\chromedriver_win32\\chromedriver.exe')
    #..that displays the map...
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    #Grab the screenshot
    browser.save_screenshot('C:\\Users\\JeffG\\Documents\\Thesis\\Grid 45 - hourly\\' + 'dist_' + str(dist_id) +"_"+ startDate.strftime('%Y-%m-%d_%H%M')+'.png')
    #Close the browser
    browser.quit()

In [ ]:
# Test function for one day
scatter_one_hour(2019,1,7,13,data,grid_id)

In [ ]:
# for i in range(29,32):
#     for j in range(0,24):
#         scatter_one_hour(2018,12,i,j,data,grid_id)

In [ ]:
# Show resulting gif
from IPython.display import Image
from IPython.display import display
display(Image(url='SPF_Eilandje_cumul.gif'))

# Plot 5 - Heatmaps per SPF
These heatmaps just show where the majority of the data points are collected per spreading factor.
In my opinion, they don't really have an added value.

In [ ]:
df.groupby('spfact').describe()

In [ ]:
# Split data into frames for the different spreading factors
class_7 = df.loc[df['spfact'] == 7][['gpsLat','gpsLon']].sample(1000).values
class_8 = df.loc[df['spfact'] == 8][['gpsLat','gpsLon']].sample(1000).values
class_9 = df.loc[df['spfact'] == 9][['gpsLat','gpsLon']].sample(1000).values
class_10 = df.loc[df['spfact'] == 10][['gpsLat','gpsLon']].sample(1000).values
class_11 = df.loc[df['spfact'] == 11][['gpsLat','gpsLon']].sample(1000).values
class_12 = df.loc[df['spfact'] == 12][['gpsLat','gpsLon']].sample(1000).values

In [ ]:
from folium.plugins import HeatMap

hm_7 = folium.Map(location=[51.214618, 4.418419], 
                    zoom_start=13)

HeatMap(class_7).add_to(hm_7)

hm_7

# Check what happens in one grid cell

### Extend the dataframe

In [24]:
def extend_df(df):
    # Sort
    df2 = df.sort_values(['device','rx_time'],ascending=[False,True]).reset_index()    
    #df2 = df.sort_values(['device'],ascending=[False]).reset_index()
    #df2 = df2.sort_values(['rx_time'],ascending=[True]).reset_index()
    
    # Add day-of-the-week columns
    df2['day_of_week'] = df2['rx_time'].dt.day_name()
    
    # Add separate time and date columns
    df2['time'] = df2['rx_time'].dt.time
    df2['date'] = df2['rx_time'].dt.date
    df2['date'] = pd.to_datetime(df2['date'])
    
    # Calculate time differences based on the device IDs 
    diff = df2.groupby('device')['rx_time'].diff()
    
    # Check if the SPF changed
    same_spfact = df2.groupby('device')['spfact'].apply(lambda x : x==x.shift())
    
    # Check how much the SPF changed
    spfact_diff = df2.groupby('device')['spfact'].apply(lambda x : x-x.shift())
    
    # Add column with SPF of previous packet
    prev_spfact = df2['spfact'].shift()
    
    # Add the new columns to the previous data frame
    df2 = df2.assign(diff=pd.Series(diff.values))
    df2['diff'] = df2['diff'].astype('timedelta64[s]')
    df2 = df2.assign(same_spfact=pd.Series(same_spfact.values))
    df2 = df2.assign(spfact_diff=pd.Series(spfact_diff.values))
    df2 = df2.assign(prev_spfact=pd.Series(prev_spfact.values))
    # Drop the rows with NaT-value
    df2 = df2.dropna(subset=['diff'])
    
    return df2

In [43]:
gdf2 = extend_df(gdf).reset_index().drop(columns=['_id','gateways','hdop','coordinates','index','level_0'])

In [26]:
# There are duplicates in the data so I should get them out first ... This doesn't do the trick
test = gdf2.drop_duplicates(inplace=True)

In [27]:
gdf2.groupby('spfact')['diff'].describe()

,count,mean,std,min,25%,50%,75%,max
spfact,,,,,,,,
7,142309.0,169.084176,5026.819126,0.0,31.0,31.0,31.0,509379.0
8,18776.0,569.353430,9941.974461,0.0,31.0,31.0,32.0,521271.0
9,13155.0,540.429799,9471.944572,0.0,31.0,31.0,38.0,508818.0
10,9174.0,436.540658,8289.177991,0.0,28.0,38.0,63.0,506183.0
11,5084.0,466.244493,9102.255621,0.0,33.0,71.0,124.0,506177.0
12,4494.0,1170.659769,50644.507779,0.0,93.0,181.0,264.0,3362138.0


## Check data where IPT<30

It's pretty weird that the minimum is zero --> This shows some duplicates
Besides that, there are quite a lot of values that are below 30s... How is this possible?

In [44]:
# Filter rows where diff < 30s
diff0 = gdf2[(gdf2['diff']<30)]
output_notebook()

Loading BokehJS ...

In [45]:
arr_hist, edges = np.histogram(diff0['diff'], 
                               bins = 30, 
                               range = [0,30])
# Put the information in a dataframe
df_filt = pd.DataFrame({'diff': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram of IPT < 30s',
          x_axis_label = 'Interpacket time (s)', 
           y_axis_label = 'Number of occurrences')

# Add a quad glyph
p.quad(bottom=0, top=df_filt['diff'], 
       left=df_filt['left'], right=df_filt['right'], 
       fill_color='red', line_color='black')

# Show the plot
show(p)

In [46]:
def scatter_plot_time(x, y):

    # create a new plot with a title and axis labels
    p = figure(title="Scatter plot", 
               x_axis_label='Time', 
               y_axis_label='Interpacket-time [s]', 
               plot_width = 800, 
               plot_height = 600,
               y_range=(-0, 30),
               x_axis_type='datetime')
    p.yaxis[0].formatter.use_scientific = False

    # add a circle renderer with a size, color, and alpha
    p.circle(x, y, size=3, color="navy", alpha=0.5)

    # show the results
    show(p)

In [47]:
scatter_plot_time(diff0['rx_time'],diff0['diff'])

In [48]:
indexes = diff0.index
diff0_prev = gdf2.iloc[indexes-1]
test = diff0.append(diff0_prev).sort_index()

## Check per grid the proportions of the different SPF in that grid

In [ ]:
def plot_barplot_SPF_grid(data, nr):
    arr_hist, edges = np.histogram(data['spfact'], 
                               bins = 6,
                              range=[6.5,12.5])
    
    # Divide the counts by the total to get a proportion
    arr_df = pd.DataFrame({'proportion': arr_hist / np.sum(arr_hist), 
                           'left': edges[:-1], 
                           'right': edges[1:] })
    
    # Format the proportion 
    arr_df['f_proportion'] = ['%0.5f' % proportion for proportion in arr_df['proportion']]

    # Create the blank plot
    p = figure(plot_height = 600, plot_width = 600, 
               title = 'Histogram of Grid {} \n # datapoints: {}'.format(nr, np.sum(arr_hist)),
                x_axis_label = 'Spreading Factor', 
               y_axis_label = 'Number of occurrences',
              y_range=(0,1))

    # Add a quad glyph
    p.quad(bottom=0, top=arr_df['proportion'], 
           left=arr_df['left'], right=arr_df['right'], 
           fill_color='red', line_color='black')

    # Show the plot
    export_png(p, filename="histograms/SPF_gridnr_{}.png".format(nr))
    #show(p)

In [ ]:
nr = 77
data = gdf2[gdf2['Area ID']==nr].reset_index()
print(len(data['spfact']))
plot_barplot_SPF_grid(data, nr)

In [ ]:
for i in range(1,101):
    # Choose one grid cell to investigate
    grid_nr = i
    data = gdf2[gdf2['Area ID']==grid_nr].reset_index()
    if len(data)!=0:
        plot_barplot_SPF_grid(data,i)

### Visualize datapoints of one grid

In [ ]:
gdf2.head()

In [ ]:
#data = gdf2[(gdf2['Area ID']==58)&(gdf2['date']=='2019-01-10')].reset_index()
data = gdf2[(gdf2['Area ID']==58)].reset_index()
scatter_plot_spf(data)

### Visualize datapoints of all areas with special distribution

In [49]:
weird_areas = [27, 36, 37, 38, 46, 47, 48, 49, 58, 61, 63 ,65, 68, 73, 74, 77, 80, 90]
plot_data = pd.DataFrame()
for i in weird_areas:
    data = gdf2[gdf2['Area ID']==i].reset_index()
    if len(data)>=50:
        size = 50
    else:
        size = len(data)
    plot_data = plot_data.append(data.sample(size))

In [50]:
scatter_plot_spf(plot_data)

## Check per grid the proportions of the different IPT in that grid

In [ ]:
def plot_barplot_IPT_grid(data, nr):
    arr_hist, edges = np.histogram(data['diff'], 
                               bins = 30,
                              range=[0,300])
    
    # Divide the counts by the total to get a proportion
    arr_df = pd.DataFrame({'proportion': arr_hist / np.sum(arr_hist), 
                           'left': edges[:-1], 
                           'right': edges[1:] })
    
    # Format the proportion 
    arr_df['f_proportion'] = ['%0.5f' % proportion for proportion in arr_df['proportion']]

    # Create the blank plot
    p = figure(plot_height = 600, plot_width = 600, 
               title = 'Histogram of Grid {} \n # datapoints: {}'.format(nr, np.sum(arr_hist)),
                x_axis_label = 'Interpacket time (s)', 
               y_axis_label = 'Number of occurrences',
              y_range=(0,1))

    # Add a quad glyph
    p.quad(bottom=0, top=arr_df['proportion'], 
           left=arr_df['left'], right=arr_df['right'], 
           fill_color='red', line_color='black')

    # Show the plot
    export_png(p, filename="histograms/IPT_gridnr_{}.png".format(nr))
    #show(p)

In [ ]:
nr = 77
data = gdf2[gdf2['Area ID']==nr].reset_index()
print(len(data['spfact']))
plot_barplot_IPT_grid(data, nr)

In [ ]:
for i in range(1,101):
    # Choose one grid cell to investigate
    grid_nr = i
    data = gdf2[gdf2['Area ID']==grid_nr].reset_index()
    if len(data)!=0:
        plot_barplot_IPT_grid(data,i)

# One van on one day

In [34]:
def scatter_plot_route(df):
    mp = folium.Map(location=[51.214618, 4.418419], 
                    zoom_start=14)

    # Generate legend
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 100px; height: 150px; 
                     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                     ">&nbsp; <b> Legend </b> <br>
                     &nbsp; SPF 7 &nbsp; <i class="fa fa-circle" style="color:darkgreen"></i><br>
                     &nbsp; SPF 8 &nbsp; <i class="fa fa-circle" style="color:green"></i><br>
                     &nbsp; SPF 9 &nbsp; <i class="fa fa-circle" style="color:yellow"></i><br>
                     &nbsp; SPF 10 &nbsp; <i class="fa fa-circle" style="color:orange"></i><br>
                     &nbsp; SPF 11 &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
                     &nbsp; SPF 12 &nbsp; <i class="fa fa-circle" style="color:darkred"></i>
          </div>
         '''
    # Add legend to map
    mp.get_root().html.add_child(folium.Element(legend_html))
    
    # Extract latitude, longitude and spreading factor
    lat = list(df["gpsLat"])
    lon = list(df["gpsLon"])
    spf = list(df["spfact"])
    time = list(df["rx_time"])
    device = list(df["device"])

    fg = folium.FeatureGroup(name="Scatter plot Spreading Factor")
    
    points = [] 
    # Add circles for every data point
    for lt, ln, sp, ti, dev in zip(lat, lon, spf, time, device):
        points.append(tuple([lt, ln]))
        cim = folium.CircleMarker(location=[lt, ln],
                                radius = 6,
                                popup="SPF: " + str(sp) + "\n timestamp: " + str(ti) + "\n device: " + str(dev),
                                fill=True, # Set fill to True
                                fill_color=color_producer(sp),
                                color = color_producer(sp),
                                fill_opacity=0.7)
        fg.add_child(cim)

    # Add line through all the points
    folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(mp)
    
    # Show the map
    mp.add_child(fg)
    
    return mp

    # Save the map
    #mp.save(os.path.join('results', 'SPF_antwerp_random.html'))
    

In [41]:
base_url = 'C:\\Users\\JeffG\\Desktop\\Case 2 - data\\Results\\routes\\'

for i in range (1, 32):
    datum = '2019-01-'+str(i)
    data = gdf2[(gdf2['device']=='343233386B376717')&(gdf2['date']==datum)].reset_index().drop(columns='index')
    data.head()
    if len(data)!=0:
        # Generate scatter plot
        mp = scatter_plot_route(data)
        
        delay=5

        #Save the map as an HTML file
        fn='route_' + datum + '.html'
        tmpurl=base_url + '{mapfile}'.format(mapfile=fn)
        print(tmpurl)
        mp.save(tmpurl)

        #Open a browser window...
        browser = webdriver.Chrome('C:\\Users\\JeffG\\Downloads\\chromedriver_win32\\chromedriver.exe')
        #..that displays the map...
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        #Grab the screenshot
        browser.save_screenshot(base_url + datum + '.png')
        #Close the browser
        browser.quit()

C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-2.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-3.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-7.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-8.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-9.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-10.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-11.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-12.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-14.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-15.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-16.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-17.html
C:\Users\JeffG\Desktop\Case 2 - data\Results\routes\route_2019-01-18.html
C:\Users\JeffG\Desktop\Case 2 - data\Result

In [42]:
scatter_plot_route(data)